In [1]:
import onnxruntime as rt
import numpy as np
import pandas as pd
import joblib

# Load the original scikit-learn model for comparison
sklearn_model = joblib.load('titanic_logistic_model.pkl')

# Load the ONNX model
onnx_model_path = "titanic_logistic_model.onnx"
sess = rt.InferenceSession(onnx_model_path)

# Create a sample input for prediction including PassengerId
sample_data = pd.DataFrame({
    'PassengerId': [892],  # Include for tracking
    'Pclass': [3],
    'Sex': [0],  # 0 for male
    'Age': [34.5],
    'SibSp': [0],
    'Parch': [0],
    'Fare': [7.8292],
    'Embarked_Q': [0],  # 0 for not Queenstown
    'Embarked_S': [1]   # 1 for Southampton
})

# Store PassengerId in a variable
passenger_id = sample_data['PassengerId'].iloc[0]

# Drop PassengerId from sample_data for prediction
sample_data = sample_data.drop(columns=['PassengerId'])

# Convert the input data to the expected format (numpy array with dtype=float32)
input_data = sample_data.to_numpy().astype(np.float32)

# Get the input and output names for the ONNX model
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

# Run inference on the ONNX model
onnx_prediction = sess.run([output_name], {input_name: input_data})

# Create a DataFrame for the output with PassengerId and Prediction
onnx_results = pd.DataFrame({
    'PassengerId': [passenger_id],  # Use the variable
    'Prediction': [onnx_prediction[0][0]]  # Take the first prediction from ONNX output
})

# Output the results from ONNX model
print("ONNX Model Prediction:")
print(onnx_results)

# For comparison, run inference using the scikit-learn model
sklearn_prediction = sklearn_model.predict(sample_data)

# Create a DataFrame for the output with PassengerId and Prediction
sklearn_results = pd.DataFrame({
    'PassengerId': [passenger_id],  # Use the variable
    'Prediction': sklearn_prediction
})

# Output the results from the scikit-learn model
print("\nScikit-learn Model Prediction:")
print(sklearn_results)


ONNX Model Prediction:
   PassengerId  Prediction
0          892           0

Scikit-learn Model Prediction:
   PassengerId  Prediction
0          892           0
